# Cleaning Data

**Prerequisites**

- [Intro](https://datascience.quantecon.org/intro.html)  
- [Boolean selection](https://datascience.quantecon.org/basics.html)  
- [Indexing](https://datascience.quantecon.org/the_index.html)  


**Outcomes**

- Be able to use string methods to clean data that comes as a string  
- Be able to drop missing data  
- Use cleaning methods to prepare and analyze a real dataset  


**Data**

- Item information from about 3,000 Chipotle meals from about 1,800
  Grubhub orders  

In [1]:
# Uncomment following line to install on colab
#! pip install qeds

In [2]:
import pandas as pd
import numpy as np
import qeds

## Outline

- [Cleaning Data](#Cleaning-Data)  
  - [Cleaning Data](#Cleaning-Data)  
  - [String Methods](#String-Methods)  
  - [Type Conversions](#Type-Conversions)  
  - [Missing Data](#Missing-Data)  
  - [Case Study](#Case-Study)  
  - [Appendix: Performance of `.str` Methods](#Appendix:-Performance-of-`.str`-Methods)  
  - [Exercises](#Exercises)  

## Cleaning Data

For many data projects, a [significant proportion of
time](https://www.forbes.com/sites/gilpress/2016/03/23/data-preparation-most-time-consuming-least-enjoyable-data-science-task-survey-says/#74d447456f63)
is spent collecting and cleaning the data — not performing the analysis.

This non-analysis work is often called “data cleaning”.

pandas provides very powerful data cleaning tools, which we
will demonstrate using the following dataset.

In [3]:
df = pd.DataFrame({"numbers": ["#23", "#24", "#18", "#14", "#12", "#10", "#35"],
                   "nums": ["23", "24", "18", "14", np.nan, "XYZ", "35"],
                   "colors": ["green", "red", "yellow", "orange", "purple", "blue", "pink"],
                   "other_column": [0, 1, 0, 2, 1, 0, 2]})
df

,numbers,nums,colors,other_column
0,#23,23,green,0
1,#24,24,red,1
2,#18,18,yellow,0
3,#14,14,orange,2
4,#12,NaN,purple,1
5,#10,XYZ,blue,0
6,#35,35,pink,2


What would happen if we wanted to try and compute the mean of
`numbers`?

```python
df["numbers"].mean()
```


It throws an error!

Can you figure out why?

Hint: When looking at error messages, start at the very
bottom.

The final error says, `TypeError: Could not convert #23#24... to numeric`.


<a id='exercise-0'></a>
> See exercise 1 in the [*exercise list*](https://datascience.quantecon.org/#exerciselist-0)

## String Methods

Our solution to the previous exercise was to remove the `#` by using
the `replace` string method: `int(c2n.replace("#", ""))`.

One way to make this change to every element of a column would be to
loop through all elements of the column and apply the desired string
methods…

In [4]:
%%time

# Iterate over all rows
for row in df.iterrows():

    # `iterrows` method produces a tuple with two elements...
    # The first element is an index and the second is a Series with the data from that row
    index_value, column_values = row

    # Apply string method
    clean_number = int(column_values["numbers"].replace("#", ""))

    # The `at` method is very similar to the `loc` method, but it is specialized
    # for accessing single elements at a time... We wanted to use it here to give
    # the loop the best chance to beat a faster method which we show you next.
    df.at[index_value, "numbers_loop"] = clean_number

CPU times: user 2.53 ms, sys: 153 µs, total: 2.68 ms
Wall time: 2.62 ms


While this is fast for a small dataset like this, this method slows for larger datasets.

One *significantly* faster (and easier) method is to apply a string
method to an entire column of data.

Most methods that are available to a Python string (we learned a
few of them in the [strings lecture](https://datascience.quantecon.org/../python_fundamentals/basics.html)) are
also available to a pandas Series that has `dtype` object.

We access them by doing `s.str.method_name` where `method_name` is
the name of the method.

When we apply the method to a Series, it is applied to all rows in the
Series in one shot!

Let’s redo our previous example using a pandas `.str` method.

In [5]:
%%time

# ~2x faster than loop... However, speed gain increases with size of DataFrame. The
# speedup can be in the ballpark of ~100-500x faster for big DataFrames.
# See appendix at the end of the lecture for an application on a larger DataFrame
df["numbers_str"] = df["numbers"].str.replace("#", "")

CPU times: user 1.09 ms, sys: 65 µs, total: 1.16 ms
Wall time: 1.13 ms


We can use `.str` to access almost any string method that works on
normal strings. (See the [official
documentation](https://pandas.pydata.org/pandas-docs/stable/text.html)
for more information.)

In [6]:
df["colors"].str.contains("p")

0    False
1    False
2    False
3    False
4     True
5    False
6     True
Name: colors, dtype: bool

In [7]:
df["colors"].str.capitalize()

0     Green
1       Red
2    Yellow
3    Orange
4    Purple
5      Blue
6      Pink
Name: colors, dtype: object


<a id='exercise-1'></a>
> See exercise 2 in the [*exercise list*](https://datascience.quantecon.org/#exerciselist-0)

## Type Conversions

In our example above, the `dtype` of the `numbers_str` column shows that pandas still treats
it as a string even after we have removed the `"#"`.

We need to convert this column to numbers.

The best way to do this is using the `pd.to_numeric` function.

This method attempts to convert whatever is stored in a Series into
numeric values

For example, after the `"#"` removed, the numbers of column
`"numbers"` are ready to be converted to actual numbers.

In [8]:
df["numbers_numeric"] = pd.to_numeric(df["numbers_str"])

In [9]:
df.dtypes

numbers             object
nums                object
colors              object
other_column         int64
numbers_loop       float64
numbers_str         object
numbers_numeric      int64
dtype: object

In [10]:
df.head()

,numbers,nums,colors,other_column,numbers_loop,numbers_str,numbers_numeric
0,#23,23,green,0,23.0,23,23
1,#24,24,red,1,24.0,24,24
2,#18,18,yellow,0,18.0,18,18
3,#14,14,orange,2,14.0,14,14
4,#12,NaN,purple,1,12.0,12,12


We can convert to other types well.

Using the `astype` method, we can convert to any of the supported
pandas `dtypes` (recall the [intro lecture](https://datascience.quantecon.org/intro.html)).

Below are some examples. (Pay attention to the reported `dtype`)

In [11]:
df["numbers_numeric"].astype(str)

0    23
1    24
2    18
3    14
4    12
5    10
6    35
Name: numbers_numeric, dtype: object

In [12]:
df["numbers_numeric"].astype(float)

0    23.0
1    24.0
2    18.0
3    14.0
4    12.0
5    10.0
6    35.0
Name: numbers_numeric, dtype: float64


<a id='exercise-2'></a>
> See exercise 3 in the [*exercise list*](https://datascience.quantecon.org/#exerciselist-0)

## Missing Data

Many datasets have missing data.

In our example, we are missing an element from the `"nums"` column.

In [13]:
df

,numbers,nums,colors,other_column,numbers_loop,numbers_str,numbers_numeric
0,#23,23,green,0,23.0,23,23
1,#24,24,red,1,24.0,24,24
2,#18,18,yellow,0,18.0,18,18
3,#14,14,orange,2,14.0,14,14
4,#12,NaN,purple,1,12.0,12,12
5,#10,XYZ,blue,0,10.0,10,10
6,#35,35,pink,2,35.0,35,35


We can find missing data by using the `isnull` method.

In [14]:
df.isnull()

,numbers,nums,colors,other_column,numbers_loop,numbers_str,numbers_numeric
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False
5,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False


We might want to know whether particular rows or columns have any
missing data.

To do this we can use the `.any` method on the boolean DataFrame
`df.isnull()`.

In [15]:
df.isnull().any(axis=0)

numbers            False
nums                True
colors             False
other_column       False
numbers_loop       False
numbers_str        False
numbers_numeric    False
dtype: bool

In [16]:
df.isnull().any(axis=1)

0    False
1    False
2    False
3    False
4     True
5    False
6    False
dtype: bool

Many approaches have been developed to deal with missing data, but the two most commonly used (and the corresponding DataFrame method) are:

- Exclusion: Ignore any data that is missing (`.dropna`).  
- Imputation: Compute “predicted” values for the data that is missing
  (`.fillna`).  


For the advantages and disadvantages of these (and other) approaches,
consider reading the [Wikipedia
article](https://en.wikipedia.org/wiki/Missing_data).

For now, let’s see some examples.

In [17]:
# drop all rows containing a missing observation
df.dropna()

,numbers,nums,colors,other_column,numbers_loop,numbers_str,numbers_numeric
0,#23,23,green,0,23.0,23,23
1,#24,24,red,1,24.0,24,24
2,#18,18,yellow,0,18.0,18,18
3,#14,14,orange,2,14.0,14,14
5,#10,XYZ,blue,0,10.0,10,10
6,#35,35,pink,2,35.0,35,35


In [18]:
# fill the missing values with a specific value
df.fillna(value=100)

,numbers,nums,colors,other_column,numbers_loop,numbers_str,numbers_numeric
0,#23,23,green,0,23.0,23,23
1,#24,24,red,1,24.0,24,24
2,#18,18,yellow,0,18.0,18,18
3,#14,14,orange,2,14.0,14,14
4,#12,100,purple,1,12.0,12,12
5,#10,XYZ,blue,0,10.0,10,10
6,#35,35,pink,2,35.0,35,35


In [19]:
# use the _next_ valid observation to fill the missing data
df.fillna(method="bfill")

,numbers,nums,colors,other_column,numbers_loop,numbers_str,numbers_numeric
0,#23,23,green,0,23.0,23,23
1,#24,24,red,1,24.0,24,24
2,#18,18,yellow,0,18.0,18,18
3,#14,14,orange,2,14.0,14,14
4,#12,XYZ,purple,1,12.0,12,12
5,#10,XYZ,blue,0,10.0,10,10
6,#35,35,pink,2,35.0,35,35


In [20]:
# use the _previous_ valid observation to fill missing data
df.fillna(method="ffill")

,numbers,nums,colors,other_column,numbers_loop,numbers_str,numbers_numeric
0,#23,23,green,0,23.0,23,23
1,#24,24,red,1,24.0,24,24
2,#18,18,yellow,0,18.0,18,18
3,#14,14,orange,2,14.0,14,14
4,#12,14,purple,1,12.0,12,12
5,#10,XYZ,blue,0,10.0,10,10
6,#35,35,pink,2,35.0,35,35


We will see more examples of dealing with missing data in future
chapters.

## Case Study

We will now use data from an
[article](https://www.nytimes.com/interactive/2015/02/17/upshot/what-do-people-actually-order-at-chipotle.html)
written by The Upshot at the NYTimes.

This data has order information from almost 2,000 Chipotle orders and
includes information on what was ordered and how much it cost.

In [21]:
chipotle = qeds.data.load("chipotle_raw")
chipotle.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98



<a id='exercise-3'></a>
> See exercise 4 in the [*exercise list*](https://datascience.quantecon.org/#exerciselist-0)

## Appendix: Performance of `.str` Methods

Let’s repeat the “remove the `#`” example from above, but this time on
a much larger dataset.

In [22]:
import numpy as np
test = pd.DataFrame({"floats": np.round(100*np.random.rand(100000), 2)})
test["strings"] = test["floats"].astype(str) + "%"
test.head()

,floats,strings
0,25.70,25.7%
1,14.43,14.43%
2,59.21,59.21%
3,53.03,53.03%
4,55.43,55.43%


In [23]:
%%time

for row in test.iterrows():
    index_value, column_values = row
    clean_number = column_values["strings"].replace("%", "")
    test.at[index_value, "numbers_loop"] = clean_number

CPU times: user 7.45 s, sys: 10 ms, total: 7.46 s
Wall time: 7.46 s


In [24]:
%%time
test["numbers_str_method"] = test["strings"].str.replace("%", "")

CPU times: user 37.7 ms, sys: 4.34 ms, total: 42.1 ms
Wall time: 40.7 ms


In [25]:
test["numbers_str_method"].equals(test["numbers_loop"])

True

We got the exact same result in a fraction of the time!

## Exercises


<a id='exerciselist-0'></a>
**Exercise 1**

Convert the string below into a number.

In [26]:
c2n = "#39" 
int(c2n.replace("#","")) #have to remove the non number elements of the string, and transform that to int

39

([*back to text*](https://datascience.quantecon.org/#exercise-0))

**Exercise 2**

Make a new column called `colors_upper` that contains the elements of
`colors` with all uppercase letters.

([*back to text*](https://datascience.quantecon.org/#exercise-1))

**Exercise 3**

Convert the column `"nums"` to a numeric type using `pd.to_numeric` and
save it to the DataFrame as `"nums_tonumeric"`.

Notice that there is a missing value, and a value that is not a number.

Look at the documentation for `pd.to_numeric` and think about how to
overcome this.

Think about why this could be a bad idea of used without
knowing what your data looks like. (Think about what happens when you
apply it to the `"numbers"` column before replacing the `"#"`.)

([*back to text*](https://datascience.quantecon.org/#exercise-2))

**Exercise 4**

We'd like you to use this data to answer the following questions.

- What is the average price of an item with chicken?  
- What is the average price of an item with steak?  
- Did chicken or steak produce more revenue (total)?  
- How many missing items are there in this dataset? How many missing
  items in each column?  


Hint: before you will be able to do any of these things you will need to
make sure the `item_price` column has a numeric `dtype` (probably
float)

([*back to text*](https://datascience.quantecon.org/#exercise-3))

Exercise 2 Answer

In [27]:
df['colors_upper'] = df["colors"].str.upper() #create new column with the information asked
print(df) #check it was created correctly

  numbers nums  colors  other_column  numbers_loop numbers_str  \
0     #23   23   green             0          23.0          23   
1     #24   24     red             1          24.0          24   
2     #18   18  yellow             0          18.0          18   
3     #14   14  orange             2          14.0          14   
4     #12  NaN  purple             1          12.0          12   
5     #10  XYZ    blue             0          10.0          10   
6     #35   35    pink             2          35.0          35   

   numbers_numeric colors_upper  
0               23        GREEN  
1               24          RED  
2               18       YELLOW  
3               14       ORANGE  
4               12       PURPLE  
5               10         BLUE  
6               35         PINK  


Exercise 3

In [28]:
df['nums_tonumeric'] = pd.to_numeric(df['nums'],errors='coerce') #use coerce to have all the column with numeric values
print(df)

  numbers nums  colors  other_column  numbers_loop numbers_str  \
0     #23   23   green             0          23.0          23   
1     #24   24     red             1          24.0          24   
2     #18   18  yellow             0          18.0          18   
3     #14   14  orange             2          14.0          14   
4     #12  NaN  purple             1          12.0          12   
5     #10  XYZ    blue             0          10.0          10   
6     #35   35    pink             2          35.0          35   

   numbers_numeric colors_upper  nums_tonumeric  
0               23        GREEN            23.0  
1               24          RED            24.0  
2               18       YELLOW            18.0  
3               14       ORANGE            14.0  
4               12       PURPLE             NaN  
5               10         BLUE             NaN  
6               35         PINK            35.0  


If coerce is used, it would delete a lot of data that might have been corrected in other ways if you had known the original data

Exercise 4

In [56]:
chipotle["item_price_num"] = chipotle["item_price"].str.replace("$","") #get data in good numeric format
chipotle["item_price_numeric"] = pd.to_numeric(chipotle["item_price_num"]) #create column from the new data
print(chipotle["item_price_numeric"].dtype) #verify correct data type for prices

float64


In [57]:
chipotle["items_with_chicken"] = chipotle["item_name"].str.contains("Chicken")
print(chipotle)

      order_id  quantity                              item_name  \
0            1         1           Chips and Fresh Tomato Salsa   
1            1         1                                   Izze   
2            1         1                       Nantucket Nectar   
3            1         1  Chips and Tomatillo-Green Chili Salsa   
4            2         2                           Chicken Bowl   
...        ...       ...                                    ...   
4617      1833         1                          Steak Burrito   
4618      1833         1                          Steak Burrito   
4619      1834         1                     Chicken Salad Bowl   
4620      1834         1                     Chicken Salad Bowl   
4621      1834         1                     Chicken Salad Bowl   

                                     choice_description item_price  \
0                                                   NaN     $2.39    
1                                          [Clementine]

In [58]:
chicken_dataframe = chipotle.loc[chipotle["item_name"].str.contains("Chicken")]
chicken_dataframe

,order_id,quantity,item_name,choice_description,item_price,item_price_num,item_price_numeric,items_with_chicken
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98,16.98,16.98,True
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98,10.98,10.98,True
11,6,1,Chicken Crispy Tacos,"[Roasted Chili Corn Salsa, [Fajita Vegetables,...",$8.75,8.75,8.75,True
12,6,1,Chicken Soft Tacos,"[Roasted Chili Corn Salsa, [Rice, Black Beans,...",$8.75,8.75,8.75,True
13,7,1,Chicken Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Rice,...",$11.25,11.25,11.25,True
...,...,...,...,...,...,...,...,...
4604,1828,1,Chicken Bowl,"[Fresh Tomato Salsa, [Rice, Black Beans, Chees...",$8.75,8.75,8.75,True
4615,1832,1,Chicken Soft Tacos,"[Fresh Tomato Salsa, [Rice, Cheese, Sour Cream]]",$8.75,8.75,8.75,True
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$11.25,11.25,11.25,True
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",$8.75,8.75,8.75,True


In [59]:
chicken_dataframe["item_price_numeric"].mean()

10.133724358974309

The average price of an item with chicken is 10.13 dollars

In [60]:
chipotle["items_with_steak"] = chipotle["item_name"].str.contains("Steak")
print(chipotle)

      order_id  quantity                              item_name  \
0            1         1           Chips and Fresh Tomato Salsa   
1            1         1                                   Izze   
2            1         1                       Nantucket Nectar   
3            1         1  Chips and Tomatillo-Green Chili Salsa   
4            2         2                           Chicken Bowl   
...        ...       ...                                    ...   
4617      1833         1                          Steak Burrito   
4618      1833         1                          Steak Burrito   
4619      1834         1                     Chicken Salad Bowl   
4620      1834         1                     Chicken Salad Bowl   
4621      1834         1                     Chicken Salad Bowl   

                                     choice_description item_price  \
0                                                   NaN     $2.39    
1                                          [Clementine]

In [61]:
steak_dataframe = chipotle.loc[chipotle["item_name"].str.contains("Steak")]
steak_dataframe

,order_id,quantity,item_name,choice_description,item_price,item_price_num,item_price_numeric,items_with_chicken,items_with_steak
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75,11.75,11.75,False,True
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25,9.25,9.25,False,True
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25,9.25,9.25,False,True
31,16,1,Steak Burrito,"[[Roasted Chili Corn Salsa (Medium), Fresh Tom...",$8.99,8.99,8.99,False,True
43,20,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Pinto Beans, Chees...",$11.75,11.75,11.75,False,True
...,...,...,...,...,...,...,...,...,...
4572,1818,1,Steak Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",$9.39,9.39,9.39,False,True
4607,1829,1,Steak Burrito,"[Tomatillo Green Chili Salsa, [Rice, Cheese, S...",$11.75,11.75,11.75,False,True
4610,1830,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",$11.75,11.75,11.75,False,True
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",$11.75,11.75,11.75,False,True


In [62]:
steak_dataframe["item_price_numeric"].mean()

10.518888888888851

The average price of an item with steak is 10.52 dollars

In [66]:
chicken_dataframe[["quantity","item_price_numeric"]]

,quantity,item_price_numeric
4,2,16.98
5,1,10.98
11,1,8.75
12,1,8.75
13,1,11.25
...,...,...
4604,1,8.75
4615,1,8.75
4619,1,11.25
4620,1,8.75


In [75]:
chipotle.count() # counts non-NaN items in each column

order_id              4622
quantity              4622
item_name             4622
choice_description    3376
item_price            4622
item_price_num        4622
item_price_numeric    4622
items_with_chicken    4622
items_with_steak      4622
dtype: int64

We see that the only column with missing items is choice_description. So, there are 1246 elements missing from the data set, all from which are missing from the choice_description column.